In [1]:
import torch
import torch.nn as nn

class selfAttention(nn.Module) :
    def __init__(self,embed_size, heads) -> None:
        '''
        embed_size : input 토큰 개수, 논문에서는 512개로 사용 
        heads : multi_head의 개수, 논문에서는 8개 사용

        Self Attention은 특정 단어(query)와 다른 단어(keys) 간의 중요도를 파악하는 매커니즘이다.
        '''

        super().__init__()


        self.embed_size = embed_size # 512차원
        self.heads = heads # 8개
        self.head_dim = embed_size // heads # 64차원(개별 attention의 차원)
        

        '''
        dict에서 쓰는 key,value 와 같다.
        query는 현재 찾고자 하는 값이다.
        '''
        # input feature, output feature
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False) 
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)

        # Multi-headed attention을 만듬
        # fully connected out 
        # input feature = outfut feature
        self.fc_out = nn.Linear(heads*self.head_dim, embed_size) # 64 * 8 
        
    def forward(self, values,keys,query,mask) :
        N = query.shape[0] # 단어 개수
        value_len = values.shape[1] # head 차원
        key_len = keys.shape[1] 
        query_len = query.shape[1]

        values = values.reshape(N,value_len, self.heads,self.head_dim)
        keys = keys.reshape(N,key_len, self.heads,self.head_dim)
        queries = query.reshape(N,query_len, self.heads,self.head_dim)

        vlaues = self.values(values) 
        keys = self.values(keys)
        queries = self.queries(queries)

        # score = Q dot K^T 
        score = torch.einsum("nqhd,nkhd->nhqk", [queries,keys]) 
        # queries shape : N,value_len, self.heads,self.head_dim
        # keys shape : N,key_len, self.heads,self.head_dim
        # score shape : N, heads, query_len, key_len
        
        # decoder 구조인 masked Self Attention 적용 시 활용되는 구문
        # score = -inf로 둬서 값을 예측 하도록 한다.
        if mask is not None :
            score = score.masked_fill(mask == 0, float("-1e20"))
            '''
            mask = 0 인 값에 대해서 -inf 대입
            -1e20 = -inf
            -inf이기 때문에 값이 0에 수렴
            mask가 부여된 경우 score 값을 0으로 준다.

            '''
        # attention 정의
        attention = torch.softmax(score / (self.embed_size**(1/2)),dim=3)

        out = torch.einsum("nhql,nlhd -> nqhd",[attention, values]).reshape(
            N,query_len,self.heads * self.head_dim
            )
        # attention shape : N, heads,query_len,key_len
        # values shape : N, value_len, heads, heads_dim
        # out shape : N, query_len, heads * head_dim

        # concat all heads 
        out = self.fc_out(out)

        return out
        

In [2]:
class TransformerBlock(nn.Module) :
    def __init__(self,embed_size, heads, dropout, forward_expansion) -> None:
        '''
        embed_size : token 개수 | 논문 512개
        heads : attention 개수 | 논문 8개
        dropout : 골고루 학습하기 위한 방법론 
        forward_expansion : forward 계산시 차원을 얼마나 늘릴 것인지 결정, 임의로 결정하는 값
                            forward_차원 계산은 forward_expension * embed_size 
                            논문에서는 4로 정함. 총 2048차원으로 늘어남.

        '''
        super().__init__()

        # Attention 정의
        self.attention = selfAttention(embed_size,heads)
        
        ### Norm & Feed Forward
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        
        self.feed_forawrd = nn.Sequential(
            # 차원을 512 -> 2048로 증가
            nn.Linear(embed_size,forward_expansion*embed_size),
            # 차원을 Relu 연산
            nn.ReLU(),
            # 차원 2048 -> 512로 축소 
            nn.Linear(forward_expansion*embed_size,embed_size)
            )
        self.dropout = nn.Dropout(dropout)

    ### Encoder 구현 
    def forward(self, value,key,query,mask) :
        # self Attention
        attention = self.attention(value, key, query, mask)
        # Add & Normalization
        x = self.dropout(self.norm1(attention + query))

        # Feed_Forward
        forward = self.feed_forawrd(x)
        # Add & Normalization
        out = self.dropout(self.norm2(forward + x))
        return out 

In [3]:
class Encoder(nn.Module) :
    def __init__(
        self, 
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
        ) -> None:
        
        '''
        src_vocab_size : input vocab 개수  
        num_layers : Encoder block 구현할 개수
        dropout : dropout 비율
        max_length : 
        '''
    
        super().__init__()

        
        self.embed_size = embed_size
        self.device = device

        # 시작부분 구현(input + positional_embeding)
        self.word_embeding = nn.Embedding(src_vocab_size, embed_size) # row / col
        self.position_embeding = nn.Embedding(max_length,embed_size) # row / col

        # Transformer Layer 구현 
        self.layers = nn.ModuleList(
            [TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion = forward_expansion,
                )
            for _ in range(num_layers)]
        )

        # dropout = 0 ~ 1
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N,seq_length = x.shape # (a,b)
        positions = torch.arange(0, seq_length).expand(N,seq_length).to(self.device) # (a,b)

        out = self.dropout(self.word_embeding(x) + self.position_embeding(positions))

        for layer in self.layers :
            # query, key, value
            out = layer(out,out,out,mask)
            
        return out 

In [4]:
class DecoderBlock(nn.Module) :
    def __init__(self,embed_size, heads, forward_expansion, dropout, device) -> None:
        super().__init__()

        self.norm = nn.LayerNorm(embed_size)
        self.attention = selfAttention(embed_size, heads=heads)
        self.transfromer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self,x,value,key,src_mask, target_mask) :
        # output에 대한 attention 수행
        attention = self.attention(x,x,x,target_mask)

        # add & Norm
        query = self.dropout(self.norm(attention + x))
        
        # encoder_decoder attention + feed_forward 
        print(value.size(),key.size(),query.size())
        out = self.transfromer_block(value, key, query, src_mask)
        return out 


class Decoder(nn.Module) :
    def __init__(
        self,
        trg_vocab_size, 
        embed_size, 
        num_layers, 
        heads, 
        forward_expansion, 
        dropout, 
        device, 
        max_length
        
        ) -> None:
        super().__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length,embed_size)

        self.layers = nn.ModuleList(
            [DecoderBlock(embed_size,heads,forward_expansion, dropout, device)
                for _ in range(num_layers)]
        )
        self.fc_out = nn.Linear(embed_size,trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x,enc_out, src_mask, trg_mask) :
        N, seq_length = x.shape
        positions = torch.arange(0,seq_length).expand(N,seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers :
            x = layer(x,enc_out, enc_out, src_mask, trg_mask)
        
        out = self.fc_out(x)
        
        return out

        

In [5]:
class transformer(nn.Module) :
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size = 512,
        num_layers = 6,
        forward_expansion = 4,
        heads = 8,
        dropout = 0,
        device = 'cpu',
        max_length = 100
    ) -> None:
    
        super().__init__()


        self.Encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(trg_vocab_size, 
            embed_size, 
            num_layers, 
            heads, 
            forward_expansion, 
            dropout, 
            device, 
            max_length)

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
    
    def mask_src_mask(self,src) :
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        #(N,1,1,src_len)
        return src_mask.to(self.device)

    def mask_trg_mask(self,trg) : 
        # trg = triangle
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(N,1,trg_len,trg_len)
        return trg_mask.to(self.device)
    
    def forward(self,src,trg) :
        src_mask = self.mask_src_mask(src)
        trg_mask = self.mask_trg_mask(trg)
        enc_src = self.Encoder(src,src_mask)
        out = self.decoder(trg,enc_src, src_mask, trg_mask)
        return out



In [14]:
### 작동 테스트
device = 'cpu'
x = torch.tensor([[1, 5, 6, 4, 3, 9, 5, 2, 0], [1, 8, 7, 3, 4, 5, 6, 7, 2]]).to(
    device
)
trg = torch.tensor([[1, 7, 4, 3, 5, 9, 2, 0], [1, 5, 6, 2, 4, 7, 6, 2]]).to(device)

src_pad_idx = 0
trg_pad_idx = 0
src_vocab_size = 100000
trg_vocab_size = 10
model = transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx, device=device).to(
    device
)
out = model(x, trg[:, :-1])
print(out.shape)

torch.Size([2, 7, 10])


In [15]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from typing import Iterable, List
from model import transformer
from data import fr_to_en
import utils_for_training as ut
import torch.nn as nn
import pandas as pd
import json
import torch

# param
SRC_LANGUAGE = "fr"
TGT_LANGUAGE = "en"

def collate_fn(batch_iter: Iterable):
    """
    Data_Loader에서 사용하는 매서드
    """
    text_transform = {}
    for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
        text_transform[ln] = ut.sequential_transforms(
            token_transfrom[ln],  # 토큰화(Tokenization)
            vocab_transform[ln],  # 수치화(Numericalization)
            ut.tensor_transform,
        )  # BOS/EOS를 추가하고 텐서를 생성
    
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch_iter:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample))

    PAD_IDX = 1
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch


### Data_load
fr_train = ut.open_text_set("data/training/train.fr")
en_train = ut.open_text_set("data/training/train.en")
vocab_transform, token_transfrom = ut.make_vocab(fr_train, en_train)


# Load_Dataset
dataset= fr_to_en(set_type='training')

# Data_loader
batch_size = 128
train_dataloader = DataLoader(dataset,batch_size,collate_fn=collate_fn)

# j = [i for i in train_dataloader]

for a,b in j[:1] :
    ''
model(a,b)

RuntimeError: The size of tensor a (128) must match the size of tensor b (100) at non-singleton dimension 1